In [18]:
import os
import torch

In [19]:
model_parent_path=os.getcwd()
model_path='google-t5/old_t5-large_qasc_epoch_0_.pkl'
# model_name='allenai/unifiedqa-v2-t5-large-1363200'
model_name=None
tokenizer_name='google-t5/t5-large'#model_name #'google-t5/t5-large'
max_answer_length = 300
BATCH_SIZE=10
accumulate_step = None
DATA_NAME_SINGLE='qasc'
NUM_EPOCHS=1
VISIBLE_DEVICE=','.join([str(x) for x in range(torch.cuda.device_count())])
DEVICE = 0

In [20]:
os.environ['CUDA_VISIBLE_DEVICES'] = VISIBLE_DEVICE #cannot work
print(f"Loi params: {model_path}{'#'*5}{max_answer_length}{'#'*5}batch_size={BATCH_SIZE};accumulate_step={accumulate_step}")
DATABASE_NAME = [DATA_NAME_SINGLE]#,'arc_hard']#,'race','mctest',]


Loi params: google-t5/old_t5-large_qasc_epoch_0_.pkl#####300#####batch_size=10;accumulate_step=None


In [21]:
import torch
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm import trange
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor
from functools import wraps, partial
from torch.nn.modules.sparse import Embedding
from torch.optim import Adam, SGD
import torch.nn as nn
from tqdm import tqdm
from random_utils import set_seed
import pandas as pd

In [22]:
# model_name = "loi_with_padding_just_same_answer_len_70.pkl"

# "loi_with_padding_1.pkl"#
# model_name = (
#     "allenai/unifiedqa-v2-t5-large-1363200"  # you can specify the model size here
# )
tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)

model_original = T5ForConditionalGeneration.from_pretrained(
    f'{model_parent_path}/{model_path}' if model_path else model_name, device_map='auto')  # 'auto')
model = model_original
# model.hf_device_map

/home/ldh0033@auburn.edu/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
NUM_QUESTION,EACH_HAS,dataset_test=pickle.load(
        open(f"multiple_choice_datasets/{DATA_NAME_SINGLE}_test_permute.pkl", "rb"))
print(NUM_QUESTION, EACH_HAS)

100 16


In [24]:
import textwrap

# ques=4
print(textwrap.fill(dataset_test[0][0]))
# print('correct: ',dataset_test[ques][1],'length',len(dataset_test[0][0]))


Climate is generally described in terms of local weather conditions.
Climate is generally described in terms of what? \n ( ) local weather
conditions ( ) Global warming ( ) occurs over a wide range ( ) measure
of motion ( ) sand ( ) forests ( ) rapid changes occur ( ) city life


In [25]:
def measure_unalike(arr):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    return 1 - ((arr / n)**2).sum()



question_to_do = 5
per_question = 20


def get_model_forward(input_ids, attention_mask, model=model):
    with torch.no_grad():
        start = []
        [start.append([0]) for x in range(len(input_ids))]
        for k in range(max_answer_length):
            # print(torch.tensor(start).shape)
            result = model(
                input_ids=input_ids.to(DEVICE),
                attention_mask=attention_mask.to(DEVICE),
                decoder_input_ids=torch.tensor(start).to(DEVICE),
                output_attentions=True,
            )
            item = result.logits.argmax(dim=2)[:, -1]
            # print('loi',result.logits.shape, item)
            for index in range(len(item)):
                start[index].append(item[index].item())
            if torch.allclose(item, torch.tensor(1)):
                break
            #     break
    result = []
    for batch in start:
        y = -1
        for index, x in enumerate(batch):
            if x == 1:
                y = index
                break
        result.append(batch[:y+1] if y > -1 else batch)
    return [tokenizer.decode(x, skip_special_tokens=True) for x in result]


def run_model(input_strs):
    if input_strs is str:
        input_strs = [input_strs]
    input_ids_wrapper = tokenizer(
        input_strs, padding=True, return_tensors='pt')

    answer = get_model_forward(input_ids_wrapper['input_ids'],
                               input_ids_wrapper['attention_mask'])
    return answer

In [26]:
run_model(dataset_test[0][0])

['local weather conditions']

In [27]:
print(NUM_QUESTION, EACH_HAS)

100 16


In [28]:
def evaluate():
    wrong_answers=[]
    got_2=[]
    got_1=[]
    answers=[]
    last_str=None
    last_acc=None
    groups=[[] for x in range(NUM_QUESTION)]
    correct=[False for x in range(NUM_QUESTION)]
    count = 0
    count1 = 0
    count2 = 0
    count10 = 0
    total = 0
    data=dataset_test
    pbar1 = trange(len(data))
    for idx, ques in enumerate(pbar1):
        groupid=int(idx /EACH_HAS)
        question = data[ques][0]
        key = data[ques][1]
        total += 1
        answer = run_model(question)[0]
        groups[groupid].append(answer)
        answers.append(answer)
        if key == answer:
            count += 1
            correct[groupid]|=True
        else:
            wrong_answers.append(ques)
        if key[0] == answer[0]:
            count1 += 1
            got_1.append(ques)
        if key[:2] == answer[:2]:
            count2 += 1
            got_2.append(ques)
        if answer in question:
            count10 += 1
        last_str=f"{count}, {count1}, {count2}, {count10},{total},{count/total*100:.2f},{count10/total*100:.2f}"
        last_acc=f'{count/total*100:.2f}'
        pbar1.set_postfix_str(last_str)
    return groups, correct, answers
groups, correct, anwers=evaluate()

100%|██████████| 1600/1600 [04:38<00:00,  5.75it/s, 1592, 1592, 1592, 1600,1600,99.50,100.00] 


### Measure consistency

In [29]:
consistency=[0 for _ in range(len(groups))]
for idx in range(len(groups)):
    consistency[idx]=1-measure_unalike(groups[idx])
    print(idx, correct[idx], consistency[idx])
    
print(f'Avg {sum(consistency)/len(consistency)}')

0 True 1.0
1 True 1.0
2 True 1.0
3 True 1.0
4 True 1.0
5 True 1.0
6 True 1.0
7 True 1.0
8 True 1.0
9 True 1.0
10 True 1.0
11 True 1.0
12 True 1.0
13 True 1.0
14 True 1.0
15 True 1.0
16 True 1.0
17 True 1.0
18 True 1.0
19 True 1.0
20 True 1.0
21 True 1.0
22 True 1.0
23 True 1.0
24 True 1.0
25 True 1.0
26 True 1.0
27 True 1.0
28 True 1.0
29 True 1.0
30 True 1.0
31 True 1.0
32 True 1.0
33 True 1.0
34 True 1.0
35 True 1.0
36 True 1.0
37 True 1.0
38 True 1.0
39 True 1.0
40 True 1.0
41 True 1.0
42 True 1.0
43 True 1.0
44 True 1.0
45 True 1.0
46 True 1.0
47 True 1.0
48 True 1.0
49 True 1.0
50 True 1.0
51 True 1.0
52 True 1.0
53 True 1.0
54 True 1.0
55 True 1.0
56 True 1.0
57 True 1.0
58 True 1.0
59 True 1.0
60 True 1.0
61 True 1.0
62 True 1.0
63 True 1.0
64 True 1.0
65 True 1.0
66 True 1.0
67 True 1.0
68 True 1.0
69 True 1.0
70 True 1.0
71 True 1.0
72 True 1.0
73 True 1.0
74 True 1.0
75 True 0.5078125
76 True 1.0
77 True 1.0
78 True 1.0
79 True 1.0
80 True 1.0
81 True 1.0
82 True 1.0
83 True 

### Measure Recall STD

In [30]:
how_many=len(dataset_test[0][0].split(' ( ) ')[1:])
total_loc=[[0 for _ in range(how_many)] for _ in range(NUM_QUESTION)]
total_acc=[[0 for _ in range(how_many)] for _ in range(NUM_QUESTION)]
for ques in range(len(dataset_test)):
    groupid=int(ques /EACH_HAS)
    key=dataset_test[ques][1]
    answers_from_text=dataset_test[ques][0].split(' ( ) ')[1:]
    location=answers_from_text.index(key)
    total_loc[groupid][location]+=1
    if anwers[ques]==key:
        total_acc[groupid][location]+=1
#         if groups[groupid][ques]==data[ques][1]:
                
# for groupid in range(len(groups)):
    

# groups[0]

### Micro

In [31]:
# import numpy as np


# recall_rate=[[x/y*100 for x,y in zip(total_acc[k], total_loc[k])] for k in range(NUM_QUESTION)]
# print(recall_rate)
# each_ques=[np.std(x) for x in recall_rate]
# print('Each ',each_ques)
# print('Final mean std',np.mean(each_ques))
# # print('std ', np.std(recall_rate))

### Macros

In [32]:
import numpy as np


print('Total',np.array(total_loc).sum(axis=0))
print('each',(np.array(total_acc).sum(axis=0)/np.array(total_loc).sum(axis=0)*100))
(np.array(total_acc).sum(axis=0)/np.array(total_loc).sum(axis=0)*100).std()

Total [200 200 200 200 200 200 200 200]
each [100.   99.  100.   99.   99.  100.   99.5  99.5]


0.4330127018922193